# FRED - Commodity Prices (Coal, LNG, Steel)

Using St. Louis Fed FRED API for global commodity price data in USD.

Data source: https://fred.stlouisfed.org/

## Python setup

In [1]:
# system imports
from pathlib import Path
from typing import Any, Dict, List
import time
import requests
from datetime import datetime, timedelta

In [2]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
# local imports
import mgplot as mg

In [4]:
# plotting setup
CHART_DIR = "./CHARTS/FRED-COMMODITIES/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()
SHOW = False
SOURCE = "Federal Reserve Economic Data (FRED)"

## FRED API Configuration

In [5]:
# FRED API Configuration
FRED_API_KEY = None  # Will be loaded from fred.api file
FRED_BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

# Load API key from file
try:
    with open("fred.api", "r") as f:
        FRED_API_KEY = f.read().strip()
    print("✅ FRED API key loaded successfully")
except FileNotFoundError:
    print("⚠️  fred.api file not found!")
    print("Please create a file called 'fred.api' in the project root containing your API key.")
    print("Get a free API key from: https://fred.stlouisfed.org/docs/api/api_key.html")
except Exception as e:
    print(f"❌ Error reading fred.api: {e}")

# Commodity price series mapping
# All prices in USD
commodity_map = {
    # Thermal Coal prices
    "PCOALAUUSDM": "Coal - Australia ($/metric ton)",
    
    # LNG and Natural Gas prices  
    "PNGASJPUSDM": "LNG - Japan ($/mmbtu)",
    "DHHNGSP": "Natural Gas - Henry Hub Spot ($/mmbtu)",
    "PNGASEUUSDM": "Natural Gas - Europe ($/mmbtu)",
    
    # Oil
    "DCOILWTICO": "Crude Oil - WTI Spot ($/barrel)",
    "DCOILBRENTEU": "Crude Oil - Brent Spot ($/barrel)",
    
    # Metals
    "PCOPPUSDM": "Copper ($/metric ton)",
    "PALUMINUMUSDM": "Aluminum ($/metric ton)",
    "PIORECRUSDM": "Iron Ore ($/metric ton)",
    "PNICKUSDM": "Nickel ($/metric ton)",
    
    # Steel
    "WPU101": "Steel Mill Products PPI (Index)",
    
    # Agriculture
    "PWHEAMTUSDM": "Wheat ($/metric ton)",
    "PMAIZMTUSDM": "Corn/Maize ($/metric ton)",
    "PSOYBUSDM": "Soybeans ($/metric ton)",
    
    # Other
    "PCU321114321114": "Lumber PPI (Index)",
    "PCOTTINDUSDM": "Cotton ($/kg)",
}

print(f"📊 Commodity Price Series: {len(commodity_map)} series")
print("💰 All prices in USD")
print("\n📈 Commodities tracked:")
for series_id, description in commodity_map.items():
    print(f"   {description}")

✅ FRED API key loaded successfully
📊 Commodity Price Series: 16 series
💰 All prices in USD

📈 Commodities tracked:
   Coal - Australia ($/metric ton)
   LNG - Japan ($/mmbtu)
   Natural Gas - Henry Hub Spot ($/mmbtu)
   Natural Gas - Europe ($/mmbtu)
   Crude Oil - WTI Spot ($/barrel)
   Crude Oil - Brent Spot ($/barrel)
   Copper ($/metric ton)
   Aluminum ($/metric ton)
   Iron Ore ($/metric ton)
   Nickel ($/metric ton)
   Steel Mill Products PPI (Index)
   Wheat ($/metric ton)
   Corn/Maize ($/metric ton)
   Soybeans ($/metric ton)
   Lumber PPI (Index)
   Cotton ($/kg)


## FRED Data Retrieval Functions

In [6]:
def detect_frequency(dates: pd.DatetimeIndex) -> str:
    """Detect the frequency of a time series from its dates.
    
    Args:
        dates: DatetimeIndex to analyze
    
    Returns:
        Frequency string ('D', 'M', 'Q', or 'A')
    """
    if len(dates) < 2:
        return 'M'  # Default to monthly if insufficient data
    
    # Try pandas infer_freq first
    freq = pd.infer_freq(dates)
    
    # Map common frequency codes to PeriodIndex-compatible ones
    if freq in ['MS', 'M', 'ME']:  # Month start, Month, Month end
        return 'M'
    elif freq in ['QS', 'Q', 'QE']:  # Quarter start, Quarter, Quarter end
        return 'Q'
    elif freq in ['AS', 'A', 'AE', 'Y', 'YS', 'YE']:  # Annual variations
        return 'A'
    elif freq in ['B', 'D']:  # Business day or daily
        return 'D'
    elif freq is not None:
        return freq
    
    # If pandas can't infer, do it manually by looking at date differences
    deltas = pd.Series(dates).diff().dt.days.dropna()
    median_days = deltas.median()
    
    if median_days <= 7:  # Daily or business daily
        return 'D'
    elif median_days <= 35:  # Monthly (allow for 28-31 day months)
        return 'M'
    elif median_days <= 100:  # Quarterly
        return 'Q'
    else:  # Annual
        return 'A'

In [7]:
def get_fred_data(series_id: str, start_date: str = "2010-01-01", api_key: str = None) -> pd.Series:
    """Get data from FRED API for a specific series.
    
    Args:
        series_id: FRED series identifier
        start_date: Start date in YYYY-MM-DD format
        api_key: FRED API key (optional if set globally)
    
    Returns:
        pandas Series with PeriodIndex and values
    """
    if api_key is None:
        api_key = FRED_API_KEY
    
    if api_key is None:
        raise ValueError("FRED API key is required. Get one from https://fred.stlouisfed.org/docs/api/api_key.html")
    
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start_date,
    }
    
    response = requests.get(FRED_BASE_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    observations = data["observations"]
    
    # Convert to pandas Series
    dates = [obs["date"] for obs in observations]
    values = [float(obs["value"]) if obs["value"] != "." else np.nan for obs in observations]
    
    series = pd.Series(values, index=pd.to_datetime(dates), name=series_id)
    series = series.dropna()
    
    # Convert to PeriodIndex with detected frequency
    if len(series) > 0:
        freq = detect_frequency(series.index)
        series.index = pd.PeriodIndex(series.index, freq=freq)
    
    return series

In [8]:
def get_commodity_prices(series_dict: Dict[str, str], start_date: str = "2010-01-01") -> Dict[str, pd.Series]:
    """Get commodity price data for multiple series from FRED.
    
    Args:
        series_dict: Dictionary mapping FRED series IDs to commodity names
        start_date: Start date for data retrieval
    
    Returns:
        Dictionary mapping commodity names to Series with PeriodIndex
    """
    price_data = {}
    successful_fetches = []
    failed_fetches = []
    
    print(f"Attempting to fetch commodity price data for {len(series_dict)} series...")
    print("="*70)
    
    for i, (series_id, commodity_name) in enumerate(series_dict.items(), 1):
        try:
            print(f"[{i:2d}/{len(series_dict)}] Fetching {commodity_name[:40]:<40} ({series_id})...", end=" ")
            
            # Get the price data
            prices = get_fred_data(series_id, start_date)
            
            # Check if we got meaningful data
            if len(prices) < 10:
                print("❌ Insufficient data")
                failed_fetches.append((commodity_name, series_id, "Insufficient data"))
                continue
            
            freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)
            print(f"✅ {len(prices)} obs ({freq_label})")
            
            prices.name = commodity_name
            price_data[commodity_name] = prices
            successful_fetches.append((commodity_name, series_id))
            
            time.sleep(0.1)  # Be nice to FRED API
            
        except Exception as e:
            error_msg = str(e)
            if "400" in error_msg:
                print("❌ Series not found")
                failed_fetches.append((commodity_name, series_id, "Series ID not found"))
            elif "403" in error_msg:
                print("❌ Access denied")
                failed_fetches.append((commodity_name, series_id, "Access denied"))
            else:
                print(f"❌ {error_msg[:30]}...")
                failed_fetches.append((commodity_name, series_id, error_msg[:50]))
            continue
    
    # Print summary
    print("\n" + "="*70)
    print("FRED DATA COLLECTION SUMMARY")
    print("="*70)
    print(f"✅ Successfully fetched: {len(successful_fetches)} series")
    print(f"❌ Failed to fetch: {len(failed_fetches)} series")
    
    if successful_fetches:
        print("\n✅ SUCCESSFUL SERIES:")
        for commodity, series_id in successful_fetches:
            print(f"   {commodity[:45]:<45} ({series_id})")
    
    if failed_fetches:
        print("\n❌ FAILED SERIES:")
        for commodity, series_id, reason in failed_fetches:
            print(f"   {commodity[:30]:<30} ({series_id}) - {reason}")
    
    return price_data

## Data Collection

In [9]:
# Collect commodity price data
if FRED_API_KEY is None:
    print("⚠️  FRED API key not loaded!")
    print("Please create a 'fred.api' file with your API key (see Setup Instructions below)")
    commodity_prices = {}
else:
    print("💰 COLLECTING COMMODITY PRICE DATA FROM FRED")
    print("Fetching commodity prices in USD...")
    print()
    
    # Get commodity price data from 2010 onwards
    commodity_prices = get_commodity_prices(commodity_map, start_date="2010-01-01")
    
    if commodity_prices:
        print(f"\n📊 DATA OVERVIEW")
        print(f"   Series: {len(commodity_prices)}")
        
        print("\n📅 LAST OBSERVATION DATE FOR EACH SERIES:")
        # Show date range for each series
        for name, series in commodity_prices.items():
            last_date = series.index[-1]
            print(f"   {name[:50]:<50} : {last_date}")

💰 COLLECTING COMMODITY PRICE DATA FROM FRED
Fetching commodity prices in USD...

Attempting to fetch commodity price data for 16 series...
[ 1/16] Fetching Coal - Australia ($/metric ton)          (PCOALAUUSDM)... 

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_58392/3876515746.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)


✅ 186 obs (monthly)
[ 2/16] Fetching LNG - Japan ($/mmbtu)                    (PNGASJPUSDM)... 

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_58392/3876515746.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)


✅ 186 obs (monthly)
[ 3/16] Fetching Natural Gas - Henry Hub Spot ($/mmbtu)   (DHHNGSP)... ✅ 4003 obs (daily)
[ 4/16] Fetching Natural Gas - Europe ($/mmbtu)           (PNGASEUUSDM)... 

...

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_58392/3876515746.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)


✅ 186 obs (monthly)
[15/16] Fetching Lumber PPI (Index)                       (PCU321114321114)... 

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_58392/3876515746.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)


✅ 188 obs (monthly)
[16/16] Fetching Cotton ($/kg)                            (PCOTTINDUSDM)... ✅ 186 obs (monthly)

FRED DATA COLLECTION SUMMARY
✅ Successfully fetched: 15 series
❌ Failed to fetch: 1 series

✅ SUCCESSFUL SERIES:
   Coal - Australia ($/metric ton)               (PCOALAUUSDM)
   LNG - Japan ($/mmbtu)                         (PNGASJPUSDM)
   Natural Gas - Henry Hub Spot ($/mmbtu)        (DHHNGSP)
   Natural Gas - Europe ($/mmbtu)                (PNGASEUUSDM)
   Crude Oil - WTI Spot ($/barrel)               (DCOILWTICO)
   Crude Oil - Brent Spot ($/barrel)             (DCOILBRENTEU)
   Copper ($/metric ton)                         (PCOPPUSDM)
   Iron Ore ($/metric ton)                       (PIORECRUSDM)
   Nickel ($/metric ton)                         (PNICKUSDM)
   Steel Mill Products PPI (Index)               (WPU101)
   Wheat ($/metric ton)                          (PWHEAMTUSDM)
   Corn/Maize ($/metric ton)                     (PMAIZMTUSDM)
   Soybeans ($/metric ton) 

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_58392/3876515746.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq_label = "daily" if prices.index.freq == 'D' else "monthly" if prices.index.freq == 'M' else str(prices.index.freq)


## Plotting Functions

In [10]:
def plot_commodity_group(data: Dict[str, pd.Series], commodity_type: str, columns: list) -> None:
    """Plot a group of related commodity prices.
    
    Args:
        data: Dictionary of commodity price series
        commodity_type: Type of commodity (for title)
        columns: List of series names to plot
    """
    if not data:
        print(f"No data to plot for {commodity_type}")
        return
    
    # Filter to available series
    available_series = {k: v for k, v in data.items() if k in columns}
    
    if not available_series:
        print(f"No data available for {commodity_type}")
        return
    
    # Create DataFrame from matching series
    plot_data = pd.DataFrame(available_series)
    
    if plot_data.empty:
        print(f"No valid data for {commodity_type}")
        return
    
    ax = mg.line_plot(plot_data, width=2, label_series=True)
    
    mg.finalise_plot(
        ax,
        title=f"{commodity_type} Prices (USD)",
        ylabel="Price (USD)",
        xlabel=None,
        rfooter=SOURCE,
        lfooter="All prices in US dollars",
        legend={"loc": "best", "fontsize": "x-small"},
        show=SHOW,
    )

In [11]:
def plot_all_commodities(data: Dict[str, pd.Series]) -> None:
    """Plot all commodities on a single chart (indexed to 100).
    
    Args:
        data: Dictionary of commodity price series
    """
    if not data:
        print("No data to plot")
        return
    
    # Index each series to 100 at the start
    indexed_data = {}
    
    for name, series in data.items():
        if len(series) > 0:
            first_value = series.iloc[0]
            indexed_data[name] = (series / first_value) * 100
    
    if not indexed_data:
        print("No data to index")
        return
    
    # Create DataFrame
    df = pd.DataFrame(indexed_data)
    
    ax = mg.line_plot(df, width=2, label_series=True)
    
    mg.finalise_plot(
        ax,
        title="Commodity Price Index (First observation = 100)",
        ylabel="Index (first observation = 100)",
        xlabel=None,
        rfooter=SOURCE,
        lfooter="All series indexed to first available observation",
        legend={"loc": "best", "fontsize": "xx-small"},
        show=SHOW,
    )

In [12]:
def plot_recent_trends(data: Dict[str, pd.Series], years: int = 5) -> None:
    """Plot recent commodity price trends.
    
    Args:
        data: Dictionary of commodity price series
        years: Number of recent years to plot
    """
    if not data:
        print("No data to plot")
        return
    
    # Find the most recent date across all series
    latest_date = max(series.index[-1].to_timestamp() for series in data.values() if len(series) > 0)
    cutoff_date = latest_date - pd.DateOffset(years=years)
    
    # Filter and index each series
    indexed_data = {}
    for name, series in data.items():
        # Filter to recent data
        recent = series[series.index.to_timestamp() >= cutoff_date]
        
        if len(recent) > 0:
            first_value = recent.iloc[0]
            indexed_data[name] = (recent / first_value) * 100
    
    if not indexed_data:
        print(f"No data available for last {years} years")
        return
    
    # Create DataFrame
    df = pd.DataFrame(indexed_data)
    
    ax = mg.line_plot(df, width=2.5, label_series=True)
    
    mg.finalise_plot(
        ax,
        title=f"Commodity Prices - Last {years} Years (Indexed)",
        ylabel=f"Index ({cutoff_date.strftime('%Y-%m')} = 100)",
        xlabel=None,
        rfooter=SOURCE,
        lfooter=f"Indexed to {cutoff_date.strftime('%Y-%m')}",
        legend={"loc": "best", "fontsize": "x-small"},
        show=SHOW,
    )

## Analysis and Plotting

In [13]:
if commodity_prices:
    print("Plotting commodity prices...")
    
    # Plot each commodity individually
    for name, series in commodity_prices.items():
        ax = mg.line_plot(series, width=2, color='darkblue')
        
        mg.finalise_plot(
            ax,
            title=name,
            ylabel="Price (USD)",
            xlabel=None,
            rfooter=SOURCE,
            lfooter=f"{len(series)} observations. Latest: {series.index[-1]}",
            show=SHOW,
        )
    
    print(f"\nPlotted {len(commodity_prices)} commodities")
    
else:
    print("No data available (FRED API key needed)")

Plotting commodity prices...

Plotted 15 commodities


## Setup Instructions

To use this notebook:

1. **Get a FRED API Key** (free):
   - Visit: https://fred.stlouisfed.org/docs/api/api_key.html
   - Create an account and request an API key

2. **Save your API key securely**:
   - Create a file named `fred.api` in the project root directory
   - Put your API key in this file (just the key, no quotes or extra text)
   - Example file contents: `a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6`

3. **Add more commodities** (optional):
   - Find FRED series IDs at https://fred.stlouisfed.org/
   - Search for commodity prices (e.g., "copper price", "oil price", "wheat price")
   - Add them to the `commodity_map` dictionary above

4. **Run the notebook**:
   - All cells should execute and produce charts with current commodity price data

**Note:** The `fred.api` file is automatically excluded from git commits for security.

## The End

In [14]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-11-17 09:08:22

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.6.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

requests  : 2.32.5
numpy     : 2.3.4
IPython   : 9.6.0
typing    : 3.10.0.0
mgplot    : 0.2.12
pathlib   : 1.0.1
matplotlib: 3.10.7
pandas    : 2.3.3

Watermark: 2.5.0

